# Práctica 2 - Inteligencia Artificial

### Grado Ingeniería Informática Tecnologías Informáticas - Curso 2019-20

### Técnicas metaheurísticas para optimización
### Algoritmos Genéticos

Miguel A. Gutérrez Naranjo - 15 de octubre, 2019.

La Gioconda es un óleo sobre tabla de álamo de 77 × 53 cm, pintado entre 1503 y 1519 por Leonardo da Vinci. En esta práctica vamos a usar algoritmos genéticos para aproximar esa imagen mediante rectángulos en escala de grises.

<img src="gioconda.jpg">

Usaremos una representación muy básica y se espera que, tras realizar la práctica, el alumno sea capaz de probar con otros polígonos y pasar de la escala de grises a imágenes en color. También es interesante buscar otras definiciones de la función decodifica, que permite generar la imagen a partir del cromosoma, otras funciones de *fitness* y otros algoritmos genéticos. En la práctica, consideraremos poblaciones de individuos, donde cada individuo se represantará mediante una tupla de genes. Además, cada individuo tendrá asociado un determinado valor *fitness* para poder comparar entre ellos. Cada gen será una tupla $(x,y,dx,dy,c)$ que interpretaremos como un rectángulo con extremo superior izquierda $(x,y)$, lados $dx$ y $dy$ y $c\in \{0,1,\dots,255\}$ representa su color (en escala de grises).

En primer lugar cargamos las librerías que vamos a necesitar.

In [ ]:
Práctica comentada y explicada por tu chavalito de confianza 😈

In [1]:
# 1.- numpy para la representación matricial
import numpy as np

# 2.- random para poder tomar valores aleatorios
import random, math

# 3.- PIL e imageio para leer y escribir imágenes.
from PIL import Image
import imageio

# Para dibujar
import matplotlib as mpl
from matplotlib import pyplot as plt
mpl.rcParams['figure.figsize'] = (15,10) # Para el tamaño de la imagen

Para empezar, tomamos la imagen que queremos aproximar desde un fichero y lo guardamos como una imagen python (en este caso el fichero gioconda.jpg). Para ello
usamos la librería PIL para convertir la imagen en una imagen en escala de grises de 8-bits (256 valores). Ver 
https://pillow.readthedocs.io/en/3.1.x/handbook/concepts.html#concept-modes

In [2]:
imagen_original = 'gioconda.jpg'
img = Image.open(imagen_original).convert('L')

Podemos ver la imagen una vez transformada

In [3]:
plt.imshow(img,cmap='gray')

A continuación, transformamos la imagen en una matriz de valores comprendido entre 0 y 255 mediante la librería numpy. Es importante saber que al cambiar el tipo de dato de imagen a matriz se cambian los ejes
https://stackoverflow.com/questions/33725237/image-fromarray-changes-size.
Llamamos IM2ARRAY a la matriz que guarda las intensidades de gris de los píxeles.

In [4]:
IM2ARRAY = np.array(img)

Podemos ver qué aspecto tiene

In [5]:
IM2ARRAY

array([[ 22,  47,  38, ...,  28,  34,  34],
       [ 87, 131,  97, ...,  26,  31,  20],
       [113, 117,  77, ...,  79,  79,  54],
       ...,
       [ 42, 102,  93, ...,  83,  77,  31],
       [ 16,  31,  17, ...,  69,  64,  31],
       [ 38,  33,  24, ...,  35,  35,  18]], dtype=uint8)

Guardamos el tamaño de la matriz en la variable *im2array_shape* para poder crear otras matrices con el mismo tamaño.

In [6]:
im2array_shape = IM2ARRAY.shape

A continuación fijamos los parámetros del algoritmo genético para aproximar la imagen original mediante rectángulos en escala de grises.

In [7]:
# Número de genes que conforman un individuo
NUMERO_DE_GENES = 150

# Tamaño de la población
TAMANO_POB = 500

# Participantes en un torneo
NUM_PARTICIPANTES = 50

# Probabilidad de mutación
PROB_MUTACION = 0.1

# Proporción de individuos que van a ser padres
PROP_CRUCES = 0.5

# Número de iteraciones
ITERACIONES = 10001

# Paso de impresión. Crearemos la imagen correspondiente al mejor individuo después de PASO_IMP iteraciones
PASO_IMP = 100

### Ejercicio 1
Define la función genera_gen() que a partir de la variable *im2array_shape* devuelva una tupla $(x,y,dx,dy,c)$. Una vez fijado el extremo $(x,y)$ dentro del lienzo, debemos tener en cuenta que el rectángulo esté dentro del lienzo y que el color esté en el rango 0-255. 

In [14]:
# Solución
def genera_gen():
    # Definimos como array los tamaños de la escala de grises de la imagen
    size_x, size_y = im2array_shape
    # Limitamos el rango de grises de la imagen tanto del eje x como del eje y aportando un valor aleatorio para cada variable
    x = random.randrange(size_x)
    y = random.randrange(size_y)
    # Sacamos la diferencia del tamaño del píxel con respecto al valor aleatorio obtenido anteriormente
    dif_x = size_x - x
    dif_y = size_y - y
    # Volvemos a sacar la diferencia en píxeles
    dx = random.randrange(dif_x)
    dy = random.randrange(dif_y)
    # Limitamos el rectángulo del lienzo hasta 255px, quedándonos con un valor
    c = random.randrange(256)
    return(x,y,dx,dy,c)

In [15]:
# Puedes probar la función
genera_gen()

# Posible respuesta:
# (185, 223, 49, 16, 2)

(112, 88, 43, 32, 112)

### Ejercicio 2
Define genera_individuo() que devuelva una tupla con tantos genes como determine el parámetro *NUMERO_DE_GENES*

In [18]:
# Solución
def genera_individuo():
    # Creamos una tupla con los valores que habrá dentro de la función
    return tuple(
        # Creamos tantos genes como el límite de NUMERO_DE_GENES nos ponga
        genera_gen() for _ in range(NUMERO_DE_GENES)
    )

In [19]:
# Ejemplo de uso. Guardamos el individuo generado en la variable ind_1
ind_1 = genera_individuo()
ind_1

# Posible respuesta:
# ((53, 10, 80, 200, 114),
#  (271, 209, 6, 4, 135),
#  ...
#  (310, 220, 35, 19, 174),
#  (264, 121, 37, 45, 200))

((209, 177, 95, 48, 226),
 (166, 89, 1, 71, 84),
 (353, 85, 5, 139, 65),
 (205, 244, 101, 0, 36),
 (123, 55, 227, 114, 174),
 (194, 76, 44, 59, 32),
 (3, 3, 248, 232, 243),
 (89, 198, 224, 15, 224),
 (335, 202, 11, 10, 107),
 (73, 133, 56, 50, 100),
 (41, 147, 127, 87, 111),
 (22, 6, 210, 157, 139),
 (259, 17, 76, 5, 217),
 (335, 60, 11, 101, 128),
 (303, 153, 10, 66, 187),
 (94, 210, 183, 23, 209),
 (97, 134, 236, 37, 214),
 (342, 50, 21, 33, 71),
 (259, 172, 48, 44, 97),
 (19, 162, 29, 13, 155),
 (66, 150, 273, 11, 168),
 (67, 188, 68, 52, 178),
 (45, 162, 100, 23, 64),
 (133, 63, 157, 6, 225),
 (65, 4, 153, 19, 39),
 (92, 232, 172, 10, 234),
 (194, 1, 38, 77, 48),
 (92, 209, 1, 34, 138),
 (204, 184, 58, 25, 127),
 (226, 100, 81, 25, 173),
 (131, 207, 155, 13, 104),
 (43, 1, 7, 155, 239),
 (119, 112, 140, 122, 166),
 (62, 203, 284, 27, 183),
 (277, 167, 6, 28, 175),
 (191, 214, 58, 12, 38),
 (98, 83, 57, 27, 207),
 (271, 200, 32, 26, 70),
 (165, 95, 169, 63, 156),
 (30, 7, 117, 25, 1

Cada gen representa un rectángulo dentro del lienzo y un individuo es una sucesión de genes. Para poder interpretar ese individuo como una imagen, primero generamos una matriz (array) a partir de ese conjunto de genes. En este caso, las posiciones que no están ocupadas por ningún cuadrado las interpretamos como blanco (valor 255) y en el resto de la posiciones se sumará la intensidad de los rectángulos que las ocupan. La función *decodifica* crea una matriz con la intensidad de gris de cada píxel a partir de un individuo.

In [20]:
def decodifica(ind):
    array_sal = np.zeros(im2array_shape,dtype='uint32')
    array_255 = np.full(im2array_shape,255,dtype='uint32')   
    for (x,y,dx,dy,c) in ind:
        array_sal[x:x+dx,y:y+dy]+= 255 - c
    mini = np.minimum(array_sal,array_255)
    inversa = 255 - mini
    return inversa

Podemos ver la matriz generada a partir del individuo que hemos creado.

In [21]:
matriz_1 = decodifica(ind_1)
print(matriz_1.tolist())

[[255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30

Podemos ver la imagen correspondiente a ese individuo transformando la matriz en una imagen

In [22]:
img_1 = Image.fromarray(matriz_1.astype('uint8'))
plt.imshow(img_1,cmap='gray')

Podemos ver una imagen junto a la otra

In [23]:
f, axarr = plt.subplots(1,2)
axarr[0].imshow(img,cmap='gray')
axarr[1].imshow(img_1,cmap='gray')

Obviamente, este individuo es una mala aproximación de la imagen original. Vamos a implementar un algoritmo genético que permita encontrar una mejor aproximación.

Para poder lanzar el algoritmo genético, en primer lugar definimos una función que nos permita crear una población inicial de individuos.

### Ejercicio 3
Define una función poblacion_inicial() que devuelva una lista con tantos individuos como determine la variable *TAMANO_POB*

In [25]:
# Solución
def poblacion_inicial():
    # Para la creación de una lista directamente usamos []
    return [
        # Definimos la población creando un individuos en cada iteración hasta que llegamos al límite de TAMANO_POB
        genera_individuo() for _ in range(TAMANO_POB)
        ]

Para poder r los mejores inidividuos de la población, necesitamos definir una función fitness. En este caso la función fitness se define como la suma de las diferencias (en valor absoluto) pixel a pixel entre la matriz que representa el individuo y la matriz que representa la imagen original.

In [26]:
def fitness(ind):
    return np.sum(np.absolute(decodifica(ind) - IM2ARRAY))

A continuación implementamos la selección por torneo. Para evitar repetir cálculos, cuando calculemos la función fitness de un individuo, la guardaremos en un diccionario y miraremos si ya ha sido calculada antes cuando necesitemos saber su valor. Las claves del diccionario serán los individuos y su valor, el valor de la función *fitness* que le corresponda.

### Ejercicio 4
Define la función *selecciona_uno_por_torneo(poblacion,dic)* que tome como entrada una población de individuos y un diccionario de pares *individuo:fitness* y devuelva una tupla *(seleccionado,nuevo_dic)* donde seleccionado sea un individuo de la población seleccionado por torneo con *NUM_PARTICIPANTES* como el número de participantes en cada torneo y *nuevo_dic* sea el diccionario *dic* al que se le han añadido los pares *individuo:fitness* que se hayan calculado en la búsqueda de *seleccionado*.

In [57]:
# Solución
def selecciona_uno_por_torneo(poblacion, dic):
    
    # Sacamos un valor infinito para usarlo como referencia el que se le pase
    minimo = float('inf')
    # Bucle limitado al número de participantes que van a ser la poblacion del cromosoma
    for _ in range(NUM_PARTICIPANTES):
        # Elegimos un indice con valor aleatorio según la población que haya como parámetro de entrada
        ind = random.choice(poblacion)
        # Una vez que vemos que el índice está en el diccionario de entrada ...
        if ind in dic:
            # ... guardamos el valor del índice como fitness índice que ya estaría previamente calculado
            f_ind = dic[ind]
        # Si el índice no está en el diccionario de entrada
        else:
            # Usamos la variable f_ind para guardar el índice del fitness
            f_ind = fitness(ind)
            # Indicamos que el valor del diccionario que apunta al índice el el fitness del índice
            dic[ind] = f_ind
            # Si el fitness del índice es menor que el mínimo actual...
            if f_ind < minimo:
                # El índice pasa a ser el fitness actual guardado en la variable actual ...
                actual = ind
                # Así cómo el mínimo pasa a ser el fitness del índice que hay ahora mismo en este individuo de la población    
                minimo = f_ind
    # La tupla se va a generar automáticamente una vez le pasemos los parámetros
    return actual, dic

In [58]:
# Ejemplo de uso
selecciona_uno_por_torneo(poblacion_inicial(),{})

(((348, 39, 14, 105, 10),
  (336, 11, 24, 118, 13),
  (191, 113, 106, 95, 13),
  (257, 129, 60, 58, 207),
  (332, 80, 14, 82, 127),
  (292, 166, 70, 68, 139),
  (51, 123, 281, 100, 108),
  (208, 166, 73, 1, 160),
  (302, 230, 52, 13, 190),
  (286, 39, 58, 102, 199),
  (127, 138, 59, 85, 184),
  (165, 119, 57, 108, 64),
  (89, 191, 274, 13, 90),
  (335, 157, 9, 49, 83),
  (316, 219, 37, 4, 139),
  (192, 228, 9, 4, 250),
  (327, 168, 34, 65, 161),
  (64, 214, 100, 30, 42),
  (67, 145, 259, 81, 151),
  (127, 234, 223, 5, 130),
  (9, 118, 95, 50, 105),
  (106, 211, 147, 12, 108),
  (165, 84, 32, 104, 194),
  (276, 231, 14, 3, 22),
  (322, 19, 10, 216, 11),
  (185, 227, 113, 17, 162),
  (234, 181, 16, 63, 26),
  (258, 77, 14, 52, 138),
  (205, 31, 143, 137, 152),
  (80, 80, 91, 62, 86),
  (26, 167, 83, 51, 60),
  (290, 170, 64, 29, 153),
  (73, 132, 160, 65, 162),
  (113, 186, 90, 48, 1),
  (318, 125, 19, 1, 136),
  (161, 232, 127, 0, 209),
  (170, 179, 46, 11, 194),
  (193, 47, 139, 109, 2

### Ejercicio 5
Define la función *seleccion_por_torneo(poblacion,num_seleccionados,dic)* que además de la población y el diccionario del ejercicio anterior toem el número de individuos que queremos seleccionar. La salida debe ser una tupla *(seleccion,nuevo_dic)* donde *nuevo_dic* es el diccionario actualizado y sleccion una lista de individuos seleccionados. 

In [36]:
# Solución
def seleccion_por_torneo(poblacion, num_seleccionados, dic):
    dic_res = dic
    # Array para la población seleccionada
    seleccion = []
    # Iteramos hasta tener un límite de valores de num_seleccionados
    for i in range(num_seleccionados):
        # Se definen las dos variables que salen como resultado de la función selecciona_uno_por_torneo( ... )
        sel, nuevo_dic = selecciona_uno_por_torneo(poblacion, dic_res)
        # El seleccionado se mete en el array de los seleccionados
        seleccion.append(sel)
        # Acumulamos el dic de la función se selección dentro de la variable resultado
        dic_res = nuevo_dic
    return seleccion, dic_res

In [37]:
# Ejemplo de uso
seleccion_por_torneo(poblacion_inicial(),4,{})

([((257, 27, 2, 3, 4),
   (356, 131, 2, 77, 81),
   (218, 31, 143, 42, 129),
   (310, 46, 33, 156, 21),
   (182, 144, 154, 32, 49),
   (308, 41, 4, 191, 204),
   (345, 212, 14, 18, 174),
   (326, 199, 23, 43, 204),
   (10, 92, 163, 41, 85),
   (3, 78, 328, 160, 181),
   (48, 233, 127, 3, 143),
   (318, 212, 31, 12, 160),
   (30, 201, 40, 32, 187),
   (27, 0, 209, 122, 105),
   (134, 54, 191, 39, 40),
   (200, 120, 157, 39, 70),
   (337, 175, 14, 30, 82),
   (294, 172, 20, 10, 32),
   (69, 33, 179, 18, 1),
   (324, 8, 0, 65, 237),
   (59, 234, 29, 1, 24),
   (279, 76, 44, 16, 112),
   (356, 30, 7, 158, 167),
   (20, 216, 29, 10, 247),
   (281, 105, 34, 65, 88),
   (241, 101, 18, 43, 3),
   (182, 123, 157, 92, 16),
   (254, 213, 1, 1, 142),
   (19, 178, 320, 61, 183),
   (198, 59, 113, 154, 141),
   (321, 176, 42, 48, 21),
   (344, 42, 14, 60, 41),
   (221, 207, 73, 35, 111),
   (129, 203, 163, 39, 127),
   (215, 186, 114, 38, 7),
   (162, 90, 198, 32, 183),
   (178, 97, 56, 74, 61),
   

A continuación definimos el cruce entre individuos

### Ejercicio 6
Define la función *cruza(i1,i2)* que tome dos individuos y devuelva una lista con los dos hijos obtenidos mediante la técnica de cruce en un punto.

In [38]:
# Solución
'''La técnica de cruce en un punto hace que para la obtención de nuevos hijos a partir de unos padres
    se de por un método aleatorio al elegir un punto de cruce, en los genes, en el que se combinan los cromosomas de 
    los padres como origen y punto de creación del nuevo hijo
    
    Que el hijo sea mejor o peor depende del fitness que tenga el propio hijo'''
def cruza(i1, i2):
    # Elegimos el punto en el que se van a cruzar los cromosomas de manera aleatoria
    # El intervalo de cruce es [1, i-1] siendo i-1 el último gen
    posicion_cruce = random.randrange(1, NUMERO_DE_GENES-1)
    """ Los crucen son la suma de cada individuo desde la posición que se escoge aleatoriamente
        por ello tenemos que cogemos los elementos del primer individuo hasta la posición del cruce elegida 
        y del segundo individuo escogemos los valores del array desde la posición de cruce hacia adelante """
    individuo_cruzado_1 = i1[posicion_cruce:] + i2[:posicion_cruce]
    individuo_cruzado_2 = i2[posicion_cruce:] + i2[:posicion_cruce]
    return(individuo_cruzado_1, individuo_cruzado_2)
    

### Ejercicio 7
Define la función *cruza_padres(padres)* que tome como entrada la lista *padres* con un número par de individuos y devuelva la lista de individuos obtenida aplicando la función *cruza(i1,i2)* donde *i1* es un individuo que ocupa una posición de índice par e *i2* es el siguiente individuo.

In [39]:
# Solución
def cruza_padres(padres):
    hijos = []
    """Este for con 3 valores (x,y,z) indica que x es el valor de inicio del bucle, es decir i=0, que y es el límite o cortocircuito del bucle
        y z es el valor incremental del valor iterador i. Sería como un for extendido en java
        for(i=0;i<hijos().size();i+2)
        
        Todo ello, porque nos dice que los hijos lo crean un PAR de padres, con lo cual se escogen los padres de 2 en 2 pero sin repetirse"""
    for i in range(0, len(padres),2):
        hijos.extend(cruza(*padres[i:i+2]))
        """Para empezar tenemos que *function hace que saquemos los individuos de la lista según lo que marquemos como límite
            En el caso de nuestro problema como ya hemos marcado vamos por pares, por ello vamos de padres[i:i+2] ya que 
            el intervalo para escoger los valores es [i, i+2) por la definición de array en python"""
        
    return hijos

En esta práctica consideraremos que la mutación de un individuo consiste en sustituir uno de sus genes por otro elegido aleatoriamente.

### Ejercicio 8
Define la función *muta(ind)* que reciba como entrada un individuo *ind* y que, con una probabilidad *PROB_MUTACION* cambie uno de sus genes por otro gen aleatorio.

In [40]:
# Solución
"""Una mutación es un cambio dentro del orden del propio cromosoma (permutación) dicha probabilidad de mutacion es baja.
    Por ello, la probabilidad aleatoria será menor que la constante que se da como entrada."""
def muta(ind):
    # Vamos a crear un booleano para que sea cierto que se va a mutar el individuo
    mutar = random.random() < PROB_MUTACION
    if mutar:
        """Vamos a escoger un gen aleatorio dentro del número de genes que hemos definido como constante, por ello el random.randrange( ... )
        Además por ello, para sustituir el valor de dicho gen, lo generaremos añadiendo el nuevo gen a la posición i aleatoria"""
        i = random.randrange(NUMERO_DE_GENES)
        return ind[i] + genera_gen() + ind[i+1:]
    else:
        return ind

### Ejercicio 9
Define una función *muta_individuos(poblacion)* que reciba como entrada a lista de individuos *poblacion* y aplique la función *muta* a cada uno de los individuos.

In [41]:
# Solución
def muta_individuos(poblacion):
    # Mutamos todos los individuos de una poblacion
    return [muta(ind) for ind in poblacion]


A continuación definimos el algoritmo que permite encontrar una nueva generación a partir de una dada.

### Ejercicio 10
Define una función *nueva_generacion(poblacion,n_padres,n_directos,dic)* que reciba como entrada:
* *poblacion* es una población de individuos
* *n_padres* es un número que determina cuántos individuos seleccionamos por torneo para ser padres
* *n_directos* es un número que determina cuántos individuos seleccionamos por torneo para pasar directamente a la siguiente generación.
* dic es un diccionario de pares individuo:fitness

La función debe seleccionar un conjunto de individuos para ser padres y otro para pasar directamente a la siguiente generación y a partir de los padres se debe genera un conjunto de hijos por cruce. La función debe devolver una tupla *(nuevo_dic,nueva_pob)* donde *nueva_pob* es una lista de individuos formada por los individuos que han pasado directamente a la siguiente generación más el resultado de aplicar la función de mutación a los hijos.

In [51]:
# Solución
def nueva_generacion(poblacion,n_padres,n_directos,dic):
    # Sacamos los padres que usaremos para cruzar la siguiente generacion y el fitness que usaremos para la generacion directa
    padres, fitness_torneo_1 = seleccion_por_torneo(poblacion, n_padres, dic)
    # Cruzamos los padres para tener unos hijos mutados con cruce
    generacion_cruce = cruza_padres(padres)
    # Generamos la generación que pasará directa por torneo usando el fitness del primer torneo
    generacion_directa, fitness_torneo_2 = seleccion_por_torneo(poblacion, n_directos, fitness_torneo_1)
    # Creamos la nueva población
    nueva_pob = generacion_directa + muta_individuos(generacion_cruce)
    # Creamos la variables nuevo_dic únicamente para seguir la nomenclatura
    nuevo_dic = fitness_torneo_2
    return nuevo_dic, nueva_pob

Por último definimos el algoritmo genetico en el que se genera un a población inicial y se van creando nuevas generaciones usando las funciones anteriores. Además, cada *PASO_IMP* imprimimos la imagen correspondiente al mejor individuo. La función devuelve la lista de los valores *fitnes*  de estos individuos para poder luego representar gráficamente la evolución de la función a lo largo de las iteraciones.

In [52]:
def algoritmo_genetico():
    poblacion= poblacion_inicial()
    dic = {}
    n_padres = round(TAMANO_POB * PROP_CRUCES)
    n_padres = (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = TAMANO_POB - n_padres
    mejores = []
    for counter in range(ITERACIONES):
        if counter%PASO_IMP == 0:
            print(counter)
            nuevo_dic = {}
            actual = 'inicial'
            min = float('inf')
            for ind in poblacion:
                f_ind = fitness(ind)
                nuevo_dic[ind] = f_ind
                if f_ind < min:
                    actual = ind
                    min = f_ind
            img_mejor = decodifica(actual).astype('uint8')
            imageio.imwrite('ga_{:>08}.jpg'.format(counter//PASO_IMP),img_mejor)
            mejores.append(min)
            dic, poblacion = nueva_generacion(poblacion,n_padres,n_directos,dic)
        else:
            dic, poblacion = nueva_generacion(poblacion,n_padres,n_directos,dic)
            print('.',end='')
    return mejores

In [56]:
sal_ag = algoritmo_genetico()

0


<class 'TypeError'>: cannot unpack non-iterable int object

In [45]:
plt.plot(sal_ag[1:])
plt.show()

<class 'NameError'>: name 'sal_ag' is not defined